### Import cell

In [1]:
import os
import sys
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import (load_nerl_data,
    get_normalized_adj,
    get_Laplace,
    calculate_random_walk_matrix,
    test_error_cap,
    test_error
)
import random
import copy
import scipy
from scipy.io import loadmat
import math
import pandas as pd
from basic_structure import D_GCN, C_GCN, K_GCN,IGNNK
from basic_process import *
from sklearn.manifold import TSNE
import os
import pandas as pd
from sklearn import linear_model
from pandas import read_csv 

2024-06-02 09:42:26.943778: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-02 09:42:27.757657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 09:42:29.552616: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from utils import *

In [3]:
n_o_n_m = 20 #sampled space dimension

h = 24 #sampled time dimension

z = 100 #hidden dimension for graph convolution

K = 1 #If using diffusion convolution, the actual diffusion convolution step is K+1

n_m = 20 #number of mask node during training

N_u = 50 #target locations, N_u locations will be deleted from the training data

Max_episode = 750 #max training episode

learning_rate = 0.0001 #the learning_rate for Adam optimizer

batch_size = 8

E_maxvalue=80

STmodel = IGNNK(h, z, K)
STmodel.load_state_dict(torch.load('model/IGNNK_ushcn_750iter_2023-11-26 12:25:53.pth'))

<All keys matched successfully>

In [30]:
#Read all data
pathroot='data/NDBC/all_stations'
PATH_ROOT = os.getcwd()
ROOT = os.path.join(PATH_ROOT, pathroot)
filenames = os.listdir(ROOT)
# Sort all files
filenames.sort()
data = []
for i in filenames:
    PATH_CSV = os.path.join(ROOT, i)
    with open(PATH_CSV, 'r') as file:
# Use splitlines() to divide contents in the documents into lists.
        content_list = file.read().splitlines()
# Transform lists into Numpy.
    content_matrix = np.array([list(map(float, line.split())) for line in content_list])
    data.append(content_matrix)
data = np.array(data).transpose(1, 2, 0)
X_raw=data

In [5]:
Station_info=pd.read_csv('data/NDBC/Station_info.csv')

### Experiment 1: Raw setting on IGNNK without training

In [6]:
for j in range(5,X_raw.shape[1]-1):
        NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
        NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
        NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
        Adj_dist=adj_dist(NDBC_lat,NDBC_long)
        X_raw_0=X_raw[:,j,:]#GET the first feature
        #print(X_raw_0.shape)
        # Count the num of missing values in each column
        missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
        # print results
       # print("Incomplete data number in each column：", missing_counts_per_column)
        #Get the index if the value is not zero
        # Find the columns where missing values exist.
        columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

        # Get the column numbers when missing value exist.
        missing_columns = np.where(columns_with_missing_data)[0]

        #Get new data after deletion.
        #Delete those columns(Stations) if there is not any features.
        result = np.delete(X_raw_0, missing_columns, axis=1)
        result = (result - np.min(result)) / (np.max(result) - np.min(result))
        #Mean results---for SNE
        mean_result_axis_1=np.mean(result,axis=0)
        mean_result_axis_1=mean_result_axis_1.reshape(-1,1)        #Get new adjacency matrix after deletion.
        NDBC_long=NDBC_long.transpose()
        NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
        # Reconstruct the index in the DataFrame and get the longitude after deletion.
        NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
        Temp_long=NDBC_long_aft.values

        NDBC_lat=NDBC_lat.transpose()
        NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
        # Reconstruct the index in the DataFrame and get the latitude after deletion.
        NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
        Temp_lat=NDBC_lat_aft.values

        merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
        tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
        array_aft = tsne.fit_transform(merged_array)

        #GEt the new ADJ matrix.
        Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
        capacities=np.ones((Adj_dist.shape[0], 1))
        capacities=capacities.flatten()
        split_line1 = int(result.shape[0] * 0.7)
        training_set = result[:split_line1, :]
        test_set = result[split_line1:, :]
        rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
        unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
        unknow_set = set(unknow_set)
        full_set = set(range(0,result.shape[1]))
        know_set = full_set - unknow_set
        training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
        A_s = Adj_dist[:, list(know_set)][list(know_set), :]
        MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
        print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

Best model in the 5 -th column is 0.40352601831183693 0.5088791407814893 -2.0018908772340307
Best model in the 6 -th column is 0.1411266781166549 0.1804560280265735 -2.0065000682516088
Best model in the 7 -th column is 0.1080074476969392 0.1418104682791045 -1.839235293152675
Best model in the 8 -th column is 0.07047534916285692 0.09903405982849459 -0.6631597810723096
Best model in the 9 -th column is 0.23394033180461932 0.2910954930464806 -2.6891267739028377
Best model in the 10 -th column is 0.12240251247475493 0.151003131106556 -2.662159706744909
Best model in the 11 -th column is 0.42374578401253 0.5127201027197579 -2.19312763841895
Best model in the 12 -th column is 0.03289810527367924 0.04290030633038398 -11.119319152080367
Best model in the 13 -th column is 0.5673032852716019 0.6638559782332828 -39.51562018379573
Best model in the 14 -th column is 0.44205008405445867 0.5270998671359972 -4.929568892482556
Best model in the 15 -th column is 0.5960648350147367 0.6273097089296188 -15

### Experiment 3.1: IGNNK-Training-dis-MI-based graph-3DGNN (Format:Metd-Adj-Only one defined graph-3DGNN)

In [7]:
#Subgraph generation module 1 --->Giving raw series, and generate a series of subgraph based on MI.
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
j=5#J-th colum, start from 5 as the first feature.
X_raw_0=X_raw[:,j,:]#GET the first feature
print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
print("Incomplete data number in each column：", missing_counts_per_column)

#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

# Print results
print("The column numbers with missing values are：", missing_columns)

# Get new data after deletion.
# Delete those columns (Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
print(result.shape)#(8784, 93)

#Determine group num here
group_num=result.shape[1]-N_u

# Initialize MI_store
MI_store = np.zeros((result.shape[1], result.shape[1]))

# Calculate mutual information
for i in range(result.shape[1]):
    for j in range(result.shape[1]):
        MI_store[i, j] = calc_MI(result[:, i], result[:, j], 10)

# Return column numbers and corresponding mutual information
column_numbers = []
MI_values = []

for i in range(MI_store.shape[0]):
    row = MI_store[i, :]
    row_index = np.arange(MI_store.shape[0])
    row_index = np.delete(row_index, i)  # Exclude the current row's index
    sorted_indices = np.argsort(row[row_index])
    max_indices = row_index[sorted_indices[-group_num:][::-1]]  # Get largest column_num
    column_numbers.append(max_indices)
    MI_values.append([row[j] for j in max_indices])

# Output results
for i, (columns, values) in enumerate(zip(column_numbers, MI_values)):
    print(f"Row {i} - Column Numbers: {columns}, MI Values: {values}")

(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175463517, 0.37428008133733837, 0.37263024544485734, 0.3321

In [8]:

from utils import load_data_MI

import torch
import numpy as np
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import *
import random
import pandas as pd
from basic_structure import IGNNK
from basic_process import *
import argparse
import sys
import os
import time
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

In [9]:
def test_error_new(STmodel, unknow_set, test_data, A_s, Missing0):
    """
    :param STmodel: The graph neural networks
    :unknow_set: The unknow locations for spatial prediction
    :test_data: The true value test_data of shape (test_num_timesteps, num_nodes)
    :A_s: The full adjacent matrix
    :Missing0: True: 0 in original datasets means missing data
    :return: NAE, MAPE and RMSE
    """
    unknow_set = set(unknow_set)
    time_dim = STmodel.time_dimension
    test_omask = np.ones(test_data.shape)
    if Missing0 == True:
        test_omask[test_data == 0] = 0
    test_inputs = (test_data * test_omask).astype('float32')
    test_inputs_s = test_inputs
    missing_index = np.ones(np.shape(test_data))
    missing_index[:, list(unknow_set)] = 0
    missing_index_s = missing_index
    o = np.zeros([test_data.shape[0]//time_dim*time_dim, test_inputs_s.shape[1]]) #Separate the test data into several h period
    for i in range(0, test_data.shape[0]//time_dim*time_dim, time_dim):
        inputs = test_inputs_s[i:i+time_dim, :]
        missing_inputs = missing_index_s[i:i+time_dim, :]
        T_inputs = inputs*missing_inputs
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        T_inputs = T_inputs/E_maxvalue
        T_inputs = np.expand_dims(T_inputs, axis = 0)
        T_inputs = torch.from_numpy(T_inputs.astype('float32'))
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_s).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_s.T).T).astype('float32'))
        imputation = STmodel(T_inputs, A_q, A_h)
        imputation = imputation.data.numpy()
        o[i:i+time_dim, :] = imputation[0, :, :]
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    if dataset == 'NREL':
        dataset=dataset
    else:
        o = o*E_maxvalue
    truth = test_inputs_s[0:test_set.shape[0]//time_dim*time_dim]
    o[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1] = truth[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1]
    test_mask =  1 - missing_index_s[0:test_set.shape[0]//time_dim*time_dim]
    if Missing0 == True:
        test_mask[truth == 0] = 0
        o[truth == 0] = 0
    MAE = np.sum(np.abs(o - truth))/np.sum( test_mask)
    RMSE = np.sqrt(np.sum((o - truth)*(o - truth))/np.sum( test_mask) )
    MAPE = np.sum(np.abs(o - truth)/(truth + 1e-5))/np.sum( test_mask)
    return MAE, RMSE, MAPE

In [10]:
# load dataset
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity = load_data_MI('ndbc')

(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [11]:

def test_error(STmodel, unknow_set, test_data, A_s, Missing0):
    """
    :param STmodel: The graph neural networks
    :unknow_set: The unknow locations for spatial prediction
    :test_data: The true value test_data of shape (test_num_timesteps, num_nodes)
    :A_s: The full adjacent matrix
    :Missing0: True: 0 in original datasets means missing data
    :return: NAE, MAPE and RMSE
    """
    unknow_set = set(unknow_set)
    time_dim = STmodel.time_dimension
    test_omask = np.ones(test_data.shape)
    if Missing0 == True:
        test_omask[test_data == 0] = 0
    test_inputs = (test_data * test_omask).astype('float32')
    test_inputs_s = test_inputs
    missing_index = np.ones(np.shape(test_data))
    missing_index[:, list(unknow_set)] = 0
    missing_index_s = missing_index
    o = np.zeros([test_data.shape[0]//time_dim*time_dim, test_inputs_s.shape[1]]) #Separate the test data into several h period
    for i in range(0, test_data.shape[0]//time_dim*time_dim, time_dim):
        inputs = test_inputs_s[i:i+time_dim, :]
        missing_inputs = missing_index_s[i:i+time_dim, :]
        T_inputs = inputs*missing_inputs
        T_inputs = T_inputs/E_maxvalue
        T_inputs = np.expand_dims(T_inputs, axis = 0)
        T_inputs = torch.from_numpy(T_inputs.astype('float32'))
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_s).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_s.T).T).astype('float32'))
        imputation = STmodel(T_inputs, A_q, A_h)
        imputation = imputation.data.numpy()
        o[i:i+time_dim, :] = imputation[0, :, :]
    if dataset == 'NREL':
        o = o*capacities[None,:]
    else:
        o = o*E_maxvalue
    truth = test_inputs_s[0:test_set.shape[0]//time_dim*time_dim]
    o[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1] = truth[missing_index_s[0:test_set.shape[0]//time_dim*time_dim] == 1]
    test_mask =  1 - missing_index_s[0:test_set.shape[0]//time_dim*time_dim]
    if Missing0 == True:
        test_mask[truth == 0] = 0
        o[truth == 0] = 0
    MAE = np.sum(np.abs(o - truth))/np.sum( test_mask)
    RMSE = np.sqrt(np.sum((o - truth)*(o - truth))/np.sum( test_mask) )
    MAPE = np.sum(np.abs(o - truth)/(truth + 1e-5))/np.sum( test_mask)
    return MAE, RMSE, MAPE

In [12]:
#torch.save(STmodel.state_dict(), '/content/Spatial-interpolation/model/IGNNK_{:}_{:}iter_{:}.pth'.format(dataset,max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
torch.save(STmodel.state_dict(), '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/IGNNK_{:}iter_{:}.pth'.format(max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
STmodel.state_dict()

OrderedDict([('GNN1.Theta1',
              tensor([[-0.0565, -0.0106,  0.0122,  ..., -0.0457, -0.0026, -0.0830],
                      [ 0.0296, -0.0353, -0.0647,  ...,  0.0538,  0.0159, -0.0188],
                      [ 0.0064,  0.0306, -0.0617,  ...,  0.0932, -0.0761, -0.0050],
                      ...,
                      [ 0.0333, -0.0776, -0.0641,  ...,  0.0298,  0.0085,  0.0693],
                      [-0.0389,  0.0677, -0.0106,  ...,  0.0133, -0.0053, -0.0038],
                      [-0.0375,  0.0726, -0.0529,  ..., -0.0637, -0.0726, -0.0040]])),
             ('GNN1.bias',
              tensor([ 0.0516, -0.0006, -0.0130,  0.0840,  0.0660,  0.0443,  0.0540, -0.0252,
                      -0.0925, -0.0581, -0.0634,  0.0417,  0.0515,  0.0052,  0.0355, -0.0728,
                       0.0627, -0.0744, -0.0446, -0.0263, -0.0756, -0.0776, -0.0634, -0.0239,
                       0.0590,  0.0439, -0.0400,  0.0145,  0.0888,  0.0073,  0.0227, -0.0540,
                      -0.0872, -0.

Inference

In [13]:
 # Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        MAE_t, RMSE_t, MAPE_t = test_error(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)


/home/xren451/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:197: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


0 6.631710590655068 8.441954593058336 1691.0599790398749
1 2.481686949929853 3.582077203818434 567.100667807699
2 1.2596293296520396 1.9339683999452697 257.6340329209771
3 1.0728840867300935 1.7137634118242497 243.92122428999502
4 1.0294897139261228 1.6493019228959025 242.6729671514905
5 0.9927324357432536 1.6000079862900578 235.65127199246461
6 0.9586074007766991 1.5532737244659205 227.81155250058254
7 0.9234267899751095 1.5022351140372483 225.21389519042265
8 0.8964682925844637 1.4613156946988535 222.38452038375348
9 0.8583292109526666 1.4036361352204048 213.77883173788067
10 0.8323424600436091 1.3580597214890684 210.70348160397765
11 0.8054945204797253 1.3129079390874598 206.39223364615918
12 0.7833383158793789 1.2725313887431606 204.17824144253476
13 0.759694907840561 1.2288437474188973 199.25658664280937
14 0.7313869900298067 1.1819071496974722 192.11105259753901
15 0.7011636779310886 1.1277626545376924 185.1460211423298
16 0.6787397125754979 1.0850322507486527 182.76406450626507


In [14]:
#Adj---Determined by tSNE
#Raw data and raw adjacency matrix without mapminmax
X_raw
# X_raw=normalize_3d_array(X_raw)

for j in range(5,X_raw.shape[1]-1):
    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    X_raw_0=X_raw[:,j,:]#GET the first feature
    #print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    # print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    #print("The column numbers with missing values are：", missing_columns)

    #Get new data after deletion.
    #Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    #Mean results---for SNE
    mean_result_axis_1=np.mean(result,axis=0)
    mean_result_axis_1=mean_result_axis_1.reshape(-1,1)

    #Get new adjacency matrix after deletion.
    NDBC_long=NDBC_long.transpose()
    NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)
    Temp_long=NDBC_long_aft.values

    NDBC_lat=NDBC_lat.transpose()
    NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    print(NDBC_lat_aft.shape)
    Temp_lat=NDBC_lat_aft.values

    merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
    tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
    array_aft = tsne.fit_transform(merged_array)

    #GEt the new ADJ matrix.
    Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
    capacities=np.ones((Adj_dist.shape[0], 1))
    capacities=capacities.flatten()
    split_line1 = int(result.shape[0] * 0.7)
    training_set = result[:split_line1, :]
    test_set = result[split_line1:, :]
    rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
    unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
    unknow_set = set(unknow_set)
    full_set = set(range(0,result.shape[1]))
    know_set = full_set - unknow_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = Adj_dist[:, list(know_set)][list(know_set), :]
    MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
    print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

(1, 93)
(1, 93)
Best model in the 5 -th column is 0.4015505003969076 0.498377812841795 -1.8792740434166402
(1, 96)
(1, 96)
Best model in the 6 -th column is 0.14810368073110042 0.18144425952082524 -2.039519230377882
(1, 94)
(1, 94)
Best model in the 7 -th column is 0.11493641726915982 0.14237521581117632 -1.8618943245312067
(1, 72)
(1, 72)
Best model in the 8 -th column is 0.0836700342933266 0.11128315262613574 -1.1000210571871114
(1, 72)
(1, 72)
Best model in the 9 -th column is 0.24296127136089493 0.28917327987332797 -2.6405662497580598
(1, 65)
(1, 65)
Best model in the 10 -th column is 0.14588280728213163 0.16714014010581282 -3.486698623011523
(1, 40)
(1, 40)
Best model in the 11 -th column is 0.4514339934759799 0.5322799320053686 -2.441404903433842
(1, 98)
(1, 98)
Best model in the 12 -th column is 0.04649584301887959 0.05097354167121652 -16.10988360663093
(1, 99)
(1, 99)
Best model in the 13 -th column is 0.5747309517393543 0.6467163210542918 -37.45053550111862
(1, 77)
(1, 77)
Bes

Experiment 3.2: IGNNK-dis-MIsubgrh-3DGNN (Format:Metd-Adj-a node can constrct multiple subgraphs-3DGNN)

In [15]:
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

In [16]:
# load dataset
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity = load_data_MI('ndbc')


(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [17]:
from __future__ import division
import torch
import numpy as np
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import *
import random
import pandas as pd
from basic_structure import IGNNK
from basic_process import *
import argparse
import sys
import os
import time
import pdb
 # Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)

0 1.3325817240188353 1.8993672093726353 920.7667793020677
1 1.0865447920887974 1.6772570508324478 815.0611899021553
2 1.016110581445984 1.5573406736224573 756.1287413837659
3 0.9297299915998647 1.4287295594767768 695.5063322744048
4 0.8471017356306052 1.3012701151089328 631.5730098718059
5 0.7767530659248479 1.1876223142122915 569.0776969957159
6 0.7034431069600716 1.081284268923408 522.7626269676612
7 0.652738496400131 0.9860728859636658 474.93799715658275
8 0.5960417457198651 0.894240903056478 430.5647560694783
9 0.5482143246062458 0.8108347183287621 391.05061319647905
10 0.5055570121900327 0.7358684535365763 357.97623255377806
11 0.4667474253670831 0.6688445140467656 326.0859835134072
12 0.44274700741895223 0.6172949951344658 303.51615169727677
13 0.41294152283285235 0.5654027021921049 276.5560453873299
14 0.38885368715477925 0.5207547814223376 263.0954349283034
15 0.373049781765865 0.4887903551347489 247.65768836640154
16 0.35265738774503264 0.45584580002211156 235.86637696516087
1

In [18]:
#torch.save(STmodel.state_dict(), '/content/Spatial-interpolation/model/IGNNK_{:}_{:}iter_{:}.pth'.format(dataset,max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
torch.save(STmodel.state_dict(), '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/model/IGNNK_{:}iter_{:}.pth'.format(max_iter,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))) # Save the model
STmodel.state_dict()

OrderedDict([('GNN1.Theta1',
              tensor([[ 0.0499,  0.0428,  0.0134,  ..., -0.0488, -0.0090,  0.0801],
                      [-0.0084,  0.0671, -0.0152,  ...,  0.0713,  0.0336,  0.0901],
                      [ 0.0803, -0.0099, -0.0407,  ...,  0.0446, -0.0782, -0.0198],
                      ...,
                      [-0.0163, -0.0456, -0.0109,  ..., -0.0085,  0.0262, -0.0321],
                      [-0.0546,  0.0290,  0.0858,  ..., -0.0898,  0.0430,  0.0034],
                      [-0.0117, -0.0630, -0.0838,  ..., -0.0557,  0.0706, -0.0893]])),
             ('GNN1.bias',
              tensor([ 0.0067,  0.0578,  0.0687,  0.0909, -0.0721,  0.0988, -0.0028,  0.0234,
                      -0.0191, -0.0805, -0.0694,  0.0541, -0.0780, -0.0075,  0.0631,  0.0589,
                       0.0463,  0.0534,  0.0498,  0.0811,  0.0719,  0.0296,  0.0509,  0.0313,
                      -0.0949,  0.0481, -0.0161,  0.0281, -0.0710,  0.0395, -0.0105,  0.0878,
                       0.0300, -0.

In [19]:
#Subgraph generation module 1 --->Giving raw series, and generate a series of subgraph based on MI.
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
j=5#J-th colum, start from 5 as the first feature.
X_raw_0=X_raw[:,j,:]#GET the first feature
print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
print("Incomplete data number in each column：", missing_counts_per_column)

#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

# Print results
print("The column numbers with missing values are：", missing_columns)

# Get new data after deletion.
# Delete those columns (Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
print(result.shape)#(8784, 93)

#Determine group num here
group_num=result.shape[1]-N_u

# Initialize MI_store
MI_store = np.zeros((result.shape[1], result.shape[1]))

# Calculate mutual information
for i in range(result.shape[1]):
    for j in range(result.shape[1]):
        MI_store[i, j] = calc_MI(result[:, i], result[:, j], 10)

# Return column numbers and corresponding mutual information
column_numbers = []
MI_values = []

for i in range(MI_store.shape[0]):
    row = MI_store[i, :]
    row_index = np.arange(MI_store.shape[0])
    row_index = np.delete(row_index, i)  # Exclude the current row's index
    sorted_indices = np.argsort(row[row_index])
    max_indices = row_index[sorted_indices[-group_num:][::-1]]  # Get largest column_num
    column_numbers.append(max_indices)
    MI_values.append([row[j] for j in max_indices])

# Output results
for i, (columns, values) in enumerate(zip(column_numbers, MI_values)):
    print(f"Row {i} - Column Numbers: {columns}, MI Values: {values}")

(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175463517, 0.37428008133733837, 0.37263024544485734, 0.3321

In [20]:
A,X,training_set,test_set,unknow_set,full_set,know_set,training_set_s,A_s,capacity,unknow_set_2D,know_set_2D,training_set_s_3D = load_data_MI_3D('ndbc')

(1, 93)
A's shape is: (93, 93)
X's shape is: (93, 8784)
(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(8784, 93)
Row 0 - Column Numbers: [ 1  2  4 79  5 20  3 77 21 17 83 85 78 87 91 12 15 31 11 43 19 44 35 30
 74 90 16 10 33 64 42 34 61 13 32 46 22 63 92 56 18  8 41], MI Values: [0.8272601132894586, 0.6901375697065202, 0.4673606483958732, 0.45757637448269683, 0.4363240338301644, 0.4190235175

In [21]:
training_set_s_3D.shape

(93, 6148, 43)

In [22]:
unknow_set_2D#List: 93*43
know_set_2D#List: 93*43

[array([ 1,  2,  4, 79,  5, 20,  3, 77, 21, 17, 83, 85, 78, 87, 91, 12, 15,
        31, 11, 43, 19, 44, 35, 30, 74, 90, 16, 10, 33, 64, 42, 34, 61, 13,
        32, 46, 22, 63, 92, 56, 18,  8, 41]),
 array([ 0, 20,  2, 85, 21, 77,  4, 79, 17,  5, 91, 87, 83,  3, 11, 15, 12,
        78, 31, 19, 10, 16, 43, 35, 30, 33, 13, 90, 64, 74, 44, 22, 42, 61,
        18, 34, 92,  8, 46, 56, 24, 28, 36]),
 array([ 4, 79,  5,  0,  3,  1, 78, 31, 20, 77, 85, 21, 17, 12, 35, 43, 87,
        83, 30, 33, 91, 74, 11, 44, 34, 10, 36, 15, 13, 86, 29, 39, 19, 42,
        37, 32, 40, 38, 89, 64, 28, 61, 41]),
 array([ 2,  5, 79,  4,  0, 84,  1, 78, 77, 31, 20, 17, 85, 83, 21, 87, 43,
        35, 12, 74, 91, 33, 44, 30, 11, 34, 64, 36, 86, 10, 46, 37, 41, 89,
        39, 32, 38, 15, 29, 42, 40, 19, 56]),
 array([ 2,  5, 79,  3,  0, 78,  1, 44, 31, 20, 77, 17, 85, 12, 87, 21, 83,
        35, 43, 11, 64, 74, 30, 33, 91, 10, 34, 63, 36, 32, 13, 19, 61, 86,
        24, 23, 38, 46, 89, 29, 15, 88, 40]),
 array([ 4

In [23]:
print(type(np.array(unknow_set_2D[0])))

<class 'numpy.ndarray'>


In [24]:
A_s

array([[1.        , 0.997613  , 0.98641608, ..., 0.73979722, 0.82128919,
        0.95039699],
       [0.997613  , 1.        , 0.97900261, ..., 0.71102215, 0.78477206,
        0.96906366],
       [0.98641608, 0.97900261, 1.        , ..., 0.82498647, 0.83981715,
        0.9097342 ],
       ...,
       [0.73979722, 0.71102215, 0.82498647, ..., 1.        , 0.81936271,
        0.57871078],
       [0.82128919, 0.78477206, 0.83981715, ..., 0.81936271, 1.        ,
        0.64313303],
       [0.95039699, 0.96906366, 0.9097342 , ..., 0.57871078, 0.64313303,
        1.        ]])

In [25]:
if __name__ == "__main__":
    """
    Model training
    """
    flags = parse_args(sys.argv[1:])
    dataset=flags.dataset
    n_o_n_m = flags.n_o
    h = flags.h
    z = flags.z
    K = flags.K
    n_m = flags.n_m
    n_u = flags.n_u
    max_iter = flags.max_iter
    learning_rate = flags.learning_rate
    E_maxvalue = flags.E_maxvalue
    batch_size = flags.batch_size
    to_plot = flags.to_plot

# Define model and train
STmodel = IGNNK(h, z, K)  # The graph neural networks
criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=learning_rate)
RMSE_list = []
MAE_list = []
MAPE_list = []
for epoch in range(max_iter):
    for i in range(training_set.shape[0]//(h * batch_size)):  #using time_length as reference to record test_error
        t_random = np.random.randint(0, high=(training_set_s.shape[0] - h), size=batch_size, dtype='l')
        know_mask = set(random.sample(range(0,training_set_s.shape[1]),n_o_n_m)) #sample n_o + n_m nodes
        feed_batch = []
        for j in range(batch_size):
            feed_batch.append(training_set_s[t_random[j]: t_random[j] + h, :][:, list(know_mask)]) #generate 8 time batches
        inputs = np.array(feed_batch)
        inputs_omask = np.ones(np.shape(inputs))
        if not dataset == 'NREL':
            inputs_omask[inputs == 0] = 0           # We found that there are irregular 0 values for METR-LA, so we treat those 0 values as missing data,
                                                    # For other datasets, it is not necessary to mask 0 values
        missing_index = np.ones((inputs.shape))
        for j in range(batch_size):
            missing_mask = random.sample(range(0,n_o_n_m),n_m) #Masked locations
            missing_index[j, :, missing_mask] = 0
        if dataset == 'NREL':
            Mf_inputs = inputs * inputs_omask * missing_index / capacities[:, None]
        else:
            Mf_inputs = inputs * inputs_omask * missing_index / E_maxvalue #normalize the value according to experience
        Mf_inputs = torch.from_numpy(Mf_inputs.astype('float32'))
        mask = torch.from_numpy(inputs_omask.astype('float32'))   #The reconstruction errors on irregular 0s are not used for training
        A_dynamic = A_s[list(know_mask), :][:, list(know_mask)]   #Obtain the dynamic adjacent matrix
        A_q = torch.from_numpy((calculate_random_walk_matrix(A_dynamic).T).astype('float32'))
        A_h = torch.from_numpy((calculate_random_walk_matrix(A_dynamic.T).T).astype('float32'))
        if dataset == 'NREL':
            outputs = torch.from_numpy(inputs/capacities[:, None])
        else:
            outputs = torch.from_numpy(inputs/E_maxvalue) #The label
        optimizer.zero_grad()
        X_res = STmodel(Mf_inputs, A_q, A_h)  #Obtain the reconstruction
        loss = criterion(X_res*mask, outputs*mask)
        loss.backward()
        optimizer.step()        #Errors backward
    if not dataset == 'NREL':
        flags = parse_args(sys.argv[1:])
        E_maxvalue = flags.E_maxvalue
        MAE_t, RMSE_t, MAPE_t = test_error_new(STmodel, unknow_set, test_set, A, E_maxvalue)
    else:
        MAE_t, RMSE_t, MAPE_t= test_error_new(STmodel, unknow_set, test_set, A,E_maxvalue)
    RMSE_list.append(RMSE_t)
    MAE_list.append(MAE_t)
    MAPE_list.append(MAPE_t)
    print(epoch, MAE_t, RMSE_t, MAPE_t)


#Adj---Determined by tSNE
#Raw data and raw adjacency matrix without mapminmax
X_raw
# X_raw=normalize_3d_array(X_raw)

for j in range(5,X_raw.shape[1]-1):
    NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
    NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
    NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
    Adj_dist=adj_dist(NDBC_lat,NDBC_long)
    X_raw_0=X_raw[:,j,:]#GET the first feature
    #print(X_raw_0.shape)
    # Count the num of missing values in each column
    missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
    # print results
    # print("Incomplete data number in each column：", missing_counts_per_column)

    #Get the index if the value is not zero
    # Find the columns where missing values exist.
    columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

    # Get the column numbers when missing value exist.
    missing_columns = np.where(columns_with_missing_data)[0]

    # Print results
    #print("The column numbers with missing values are：", missing_columns)

    #Get new data after deletion.
    #Delete those columns(Stations) if there is not any features.
    result = np.delete(X_raw_0, missing_columns, axis=1)
    result = (result - np.min(result)) / (np.max(result) - np.min(result))
    #Mean results---for SNE
    mean_result_axis_1=np.mean(result,axis=0)
    mean_result_axis_1=mean_result_axis_1.reshape(-1,1)

    #Get new adjacency matrix after deletion.
    NDBC_long=NDBC_long.transpose()
    NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the longitude after deletion.
    NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
    print(NDBC_long_aft.shape)
    Temp_long=NDBC_long_aft.values

    NDBC_lat=NDBC_lat.transpose()
    NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
    # Reconstruct the index in the DataFrame and get the latitude after deletion.
    NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
    print(NDBC_lat_aft.shape)
    Temp_lat=NDBC_lat_aft.values

    merged_array = np.concatenate([mean_result_axis_1, Temp_long.T,Temp_lat.T], axis=1)
    tsne = TSNE(n_components=2, random_state=30,perplexity=1) #Make sure random seed to be the same for the reproducibility.
    array_aft = tsne.fit_transform(merged_array)

    #GEt the new ADJ matrix.
    Adj_dist=adj_dist(pd.DataFrame(array_aft[:,0]),pd.DataFrame(array_aft[:,1]))
    capacities=np.ones((Adj_dist.shape[0], 1))
    capacities=capacities.flatten()
    split_line1 = int(result.shape[0] * 0.7)
    training_set = result[:split_line1, :]
    test_set = result[split_line1:, :]
    rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
    unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=True)#Stations to be predicted
    unknow_set = set(unknow_set)
    full_set = set(range(0,result.shape[1]))
    know_set = full_set - unknow_set
    training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
    A_s = Adj_dist[:, list(know_set)][list(know_set), :]
    MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
    print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

0 2.9203556402193063 3.913276203829703 1280.1550705929392
1 1.0844112887328354 1.7650342739236047 691.1089228782006
2 1.0235080604431586 1.6870539664633062 668.0174898895349
3 0.9787655436271351 1.6271454489187078 636.8251430059246
4 0.9413413908005858 1.56180242474432 610.7190276224878
5 0.8995009456479809 1.4962592322051735 580.1774435278503
6 0.862458628921053 1.4401765996027753 553.2233360836184
7 0.8293900221806544 1.380330509083718 526.085454851753
8 0.8014650396906984 1.3280572620662732 503.1710039334792
9 0.7750167535146786 1.2770531131449565 482.25655114310456
10 0.7377263365269995 1.2181752602502252 458.3446411503347
11 0.7157558356609963 1.1690594934719047 442.5090903787714
12 0.6845529991302255 1.1153294344946525 422.29874290568654
13 0.6560751747255998 1.0580458964911346 398.8872555356028
14 0.6264829597628475 1.0123335883609013 381.9871838992194
15 0.6032151093886933 0.9648402213585733 361.0735883342919
16 0.5755705724545276 0.9156554323812952 341.6902113508222
17 0.55663

In [26]:
##Generate the code based on the GMOE+GCN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch_geometric.nn import GCNConv

# Define the Graph Convolutional Layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj_matrix):
        out = torch.matmul(adj_matrix, x)  # Aggregate neighborhood information
        out = self.linear(out)  # Linear transformation
        return out

# Define the Mixture of Experts Model
class GMoE(nn.Module):
    def __init__(self, num_experts, in_features, out_features):
        super(GMoE, self).__init__()
        self.num_experts = num_experts
        self.experts = nn.ModuleList([GraphConvolution(in_features, out_features) for _ in range(num_experts)])
        self.gate = nn.Linear(in_features, num_experts)

    def forward(self, x, adj_matrix):
        gate_outputs = torch.sigmoid(self.gate(x))  # Compute expert selection gate outputs
        expert_outputs = [expert(x, adj_matrix) for expert in self.experts]  # Compute expert outputs
        mixed_output = torch.stack(expert_outputs, dim=-1)  # Stack expert outputs along the last dimension
        weighted_output = torch.matmul(mixed_output, gate_outputs.unsqueeze(-1))  # Weighted sum of expert outputs
        return weighted_output.squeeze(-1)  # Squeeze out the last dimension

# Example usage:
# Generate random adjacency matrix for subgraphs
def generate_adj_matrix(num_nodes):
    # Generate a random adjacency matrix with values between 0 and 1
    adj_matrix = torch.rand(num_nodes, num_nodes)
    # Make the adjacency matrix symmetric
    adj_matrix = (adj_matrix + adj_matrix.t()) / 2
    # Normalize the adjacency matrix
    adj_matrix = adj_matrix / adj_matrix.sum(dim=1, keepdim=True)
    return adj_matrix

# Generate random input features and target labels
num_nodes = 10
num_features = 5
num_targets = 1
num_experts = 3
num_samples = 100
input_features = torch.randn(num_samples, num_nodes, num_features)
target_labels = torch.randn(num_samples, num_nodes, num_targets)

# Initialize the MoE model
moe_model = GMoE(num_experts, num_features, num_targets)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(moe_model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    for i in range(num_samples):
        adj_matrix = generate_adj_matrix(num_nodes)
        outputs = moe_model(input_features[i], adj_matrix)
        loss = criterion(outputs, target_labels[i])
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, you can use the trained MoE model for spatial interpolation tasks.

Epoch [1/100], Loss: 0.4847
Epoch [2/100], Loss: 0.1391
Epoch [3/100], Loss: 0.1705
Epoch [4/100], Loss: 0.1619
Epoch [5/100], Loss: 0.1278
Epoch [6/100], Loss: 0.1510
Epoch [7/100], Loss: 0.1292
Epoch [8/100], Loss: 0.1702
Epoch [9/100], Loss: 0.1241
Epoch [10/100], Loss: 0.1725
Epoch [11/100], Loss: 0.1682
Epoch [12/100], Loss: 0.1890
Epoch [13/100], Loss: 0.1529
Epoch [14/100], Loss: 0.1454
Epoch [15/100], Loss: 0.1409
Epoch [16/100], Loss: 0.2227
Epoch [17/100], Loss: 0.1580
Epoch [18/100], Loss: 0.1320
Epoch [19/100], Loss: 0.1530
Epoch [20/100], Loss: 0.1508
Epoch [21/100], Loss: 0.1567
Epoch [22/100], Loss: 0.1650
Epoch [23/100], Loss: 0.1855
Epoch [24/100], Loss: 0.1454
Epoch [25/100], Loss: 0.1490
Epoch [26/100], Loss: 0.1454
Epoch [27/100], Loss: 0.1577
Epoch [28/100], Loss: 0.1164
Epoch [29/100], Loss: 0.1539
Epoch [30/100], Loss: 0.1277
Epoch [31/100], Loss: 0.1548
Epoch [32/100], Loss: 0.1224
Epoch [33/100], Loss: 0.1706
Epoch [34/100], Loss: 0.1662
Epoch [35/100], Loss: 0

In [27]:
adj_matrix.shape

torch.Size([10, 10])

In [28]:
##Generate the code based on the GMOE+GCN+GAT
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATConv
from torch_geometric.utils import to_undirected, add_self_loops

# Define the Graph Convolutional Layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, edge_index):
        out = self.linear(x)  # Linear transformation
        return out

# Define the Mixture of Experts Model
class GMoE(nn.Module):
    def __init__(self, num_experts, in_features, out_features):
        super(GMoE, self).__init__()
        self.num_experts = num_experts
        self.experts = nn.ModuleList([
            GraphConvolution(in_features, out_features) for _ in range(num_experts // 2)
        ])
        self.experts.extend([
            GATConv(in_features, out_features, heads=1) for _ in range(num_experts // 2)
        ])
        self.gate = nn.Linear(in_features, num_experts)

    def forward(self, x, edge_index):
        gate_outputs = torch.sigmoid(self.gate(x))  # Compute expert selection gate outputs
        expert_outputs = [expert(x, edge_index) for expert in self.experts]  # Compute expert outputs
        mixed_output = torch.stack(expert_outputs, dim=-1)  # Stack expert outputs along the last dimension
        weighted_output = torch.matmul(mixed_output, gate_outputs.unsqueeze(-1))  # Weighted sum of expert outputs
        return weighted_output.squeeze(-1)  # Squeeze out the last dimension

# Example usage:
# Generate random edge indices for subgraphs
def generate_edge_index(num_nodes):
    edge_index = torch.randint(0, num_nodes, (2, num_nodes))  # Assuming a complete graph
    edge_index = to_undirected(edge_index)
    edge_index, _ = add_self_loops(edge_index)
    return edge_index

# Generate random input features and target labels
num_nodes = 10
num_features = 5
num_targets = 1
num_experts = 4  # Divisible by 2
num_samples = 100
input_features = torch.randn(num_samples, num_nodes, num_features)
target_labels = torch.randn(num_samples, num_nodes, num_targets)

# Initialize the MoE model
moe_model = GMoE(num_experts, num_features, num_targets)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(moe_model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    for i in range(num_samples):
        edge_index = generate_edge_index(num_nodes)
        outputs = moe_model(input_features[i], edge_index)
        loss = criterion(outputs, target_labels[i])
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 3.0433
Epoch [2/100], Loss: 1.0337
Epoch [3/100], Loss: 1.0992
Epoch [4/100], Loss: 1.5075
Epoch [5/100], Loss: 1.1313
Epoch [6/100], Loss: 0.9354
Epoch [7/100], Loss: 0.9507
Epoch [8/100], Loss: 0.8557
Epoch [9/100], Loss: 0.8620
Epoch [10/100], Loss: 0.8666
Epoch [11/100], Loss: 0.8445
Epoch [12/100], Loss: 0.7874
Epoch [13/100], Loss: 0.8413
Epoch [14/100], Loss: 0.8609
Epoch [15/100], Loss: 0.8321
Epoch [16/100], Loss: 0.7965
Epoch [17/100], Loss: 0.7729
Epoch [18/100], Loss: 0.8059
Epoch [19/100], Loss: 0.7795
Epoch [20/100], Loss: 0.8681
Epoch [21/100], Loss: 0.8453
Epoch [22/100], Loss: 0.9570
Epoch [23/100], Loss: 0.8482
Epoch [24/100], Loss: 0.8779
Epoch [25/100], Loss: 0.9236
Epoch [26/100], Loss: 0.8548
Epoch [27/100], Loss: 0.8759
Epoch [28/100], Loss: 0.8259
Epoch [29/100], Loss: 0.9231
Epoch [30/100], Loss: 0.8043
Epoch [31/100], Loss: 0.8421
Epoch [32/100], Loss: 0.8480
Epoch [33/100], Loss: 0.8154
Epoch [34/100], Loss: 0.9147
Epoch [35/100], Loss: 0

In [29]:
##Some errors happen when we try to use GMOE+GCN+GAT+VAE
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATConv
from torch_geometric.utils import to_undirected, add_self_loops

# Define the Graph Convolutional Layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, edge_index):
        out = self.linear(x)  # Linear transformation
        return out

# Define the Variational Autoencoder
class VariationalAutoencoder(nn.Module):
    def __init__(self, in_features, latent_features, num_targets, num_nodes):
        super(VariationalAutoencoder, self).__init__()
        self.num_nodes = num_nodes
        self.num_targets = num_targets
        # Define your variational autoencoder architecture here
        # For example:
        self.encoder = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, latent_features * 2)  # *2 for mean and variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_features, 128),
            nn.ReLU(),
            nn.Linear(128, num_nodes * num_targets)  # Adjusted output size
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def pad_or_reshape_output(self, output):
        batch_size = output.size(0)
        if output.size(1) != self.num_nodes or output.size(2) != self.num_targets:
            # Pad or reshape the output tensor to match the desired shape
            if output.size(1) < self.num_nodes or output.size(2) < self.num_targets:
                # Pad the output tensor with zeros
                padded_output = torch.zeros(batch_size, self.num_nodes, self.num_targets)
                padded_output[:, :output.size(1), :output.size(2)] = output
                return padded_output
            else:
                # Reshape the output tensor
                reshaped_output = output[:, :self.num_nodes, :self.num_targets]
                return reshaped_output
        else:
            return output

    def forward(self, x, edge_index=None):
        # Forward pass through encoder
        z_mu_log_var = self.encoder(x)
        z_mu, z_log_var = z_mu_log_var.chunk(2, dim=-1)
        z = self.reparameterize(z_mu, z_log_var)
        # Forward pass through decoder
        output = self.decoder(z).view(-1, self.num_nodes, self.num_targets)  # Reshape to match expert output size
        if edge_index is not None:
            output = self.pad_or_reshape_output(output)
        return output


# Define the Mixture of Experts Model
class GMoE(nn.Module):
    def __init__(self, num_experts, in_features, out_features, latent_features, num_targets, num_nodes):
        super(GMoE, self).__init__()
        self.num_experts = num_experts
        self.experts = nn.ModuleList([
            GraphConvolution(in_features, out_features) for _ in range(num_experts // 3)
        ])
        self.experts.extend([
            GATConv(in_features, out_features, heads=1) for _ in range(num_experts // 3)
        ])
        self.experts.extend([
            VariationalAutoencoder(in_features, latent_features, num_targets, num_nodes) for _ in range(num_experts // 3)
        ])
        self.gate = nn.Linear(in_features, num_experts)

    def forward(self, x, edge_index):
        gate_outputs = torch.sigmoid(self.gate(x))  # Compute expert selection gate outputs
        
        # Compute expert outputs and store their sizes
        expert_outputs = [expert(x, edge_index) for expert in self.experts]  # Compute expert outputs
        
        # Find the maximum size along node and target dimensions
        max_num_nodes = max(output.size(-2) for output in expert_outputs)
        max_num_targets = max(output.size(-1) for output in expert_outputs)
        
        # Pad expert outputs to match maximum size
        padded_outputs = []
        for output in expert_outputs:
            padding_nodes = max_num_nodes - output.size(-2)
            padding_targets = max_num_targets - output.size(-1)
            padded_output = nn.functional.pad(output, (0, padding_targets, 0, padding_nodes))
            padded_outputs.append(padded_output)

        # Stack padded expert outputs along the last dimension
        mixed_output = torch.stack(padded_outputs, dim=-1)  # Stack expert outputs along the last dimension
        
        # Weighted sum of expert outputs
        weighted_output = torch.matmul(mixed_output, gate_outputs.unsqueeze(-1))
        
        return weighted_output.squeeze(-1)  # Squeeze out the last dimension

# Example usage:
# Generate random edge indices for subgraphs
def generate_edge_index(num_nodes):
    edge_index = torch.randint(0, num_nodes, (2, num_nodes))  # Assuming a complete graph
    edge_index = to_undirected(edge_index)
    edge_index, _ = add_self_loops(edge_index)
    return edge_index

# Generate random input features and target labels
num_nodes = 10
num_features = 5
num_targets = 1
num_experts = 6  # Divisible by 3
num_samples = 100
input_features = torch.randn(num_samples, num_nodes, num_features)
target_labels = torch.randn(num_samples, num_nodes, num_targets)

# Initialize the MoE model
latent_features = 10  # Dimensionality of the latent space representation
moe_model = GMoE(num_experts, num_features, num_targets, latent_features, num_targets, num_nodes)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(moe_model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    for i in range(num_samples):
        edge_index = generate_edge_index(num_nodes)
        outputs = moe_model(input_features[i], edge_index)
        loss = criterion(outputs, target_labels[i])
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


RuntimeError: stack expects each tensor to be equal size, but got [10, 1] at entry 0 and [10, 10, 1] at entry 4

In [ ]:
!pip install spektral

In [ ]:
import spektral
channels=2
spektral.layers.DiffusionConv(channels, K=6, activation='tanh', kernel_initializer='glorot_uniform', kernel_regularizer=None, kernel_constraint=None)
x_try=[2,3,2,4]
y_try=[3,4,2,1]
y_test=spektral.layers.DiffusionConv(x_try)

In [ ]:
y_test

In [ ]:
#Some error happens when we try to use KalmanFilter+GCN
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Define the Kalman Filter
class KalmanFilter(nn.Module):
    def __init__(self, num_features, num_nodes):
        super(KalmanFilter, self).__init__()
        self.num_features = num_features
        self.num_nodes = num_nodes
        self.observation_matrix = nn.Linear(num_features, num_nodes * num_features, bias=False)
        self.transition_matrix = nn.Linear(num_features, num_features, bias=False)
        self.process_noise_covariance = nn.Parameter(torch.Tensor(num_features, num_features))
        self.observation_noise_covariance = nn.Parameter(torch.Tensor(num_nodes * num_features, num_nodes * num_features))
        self.hidden_state_mean = nn.Parameter(torch.zeros(num_nodes, num_features))
        self.hidden_state_covariance = nn.Parameter(torch.eye(num_nodes * num_features))

    def forward(self, observations):
        num_samples = observations.size(0)
        predicted_hidden_states = []
        hidden_state = self.hidden_state_mean.unsqueeze(0).repeat(num_samples, 1, 1)
        for observation in observations:
            observation = observation.transpose(0, 1).contiguous()  # Transpose to (num_features, num_samples)
            observation = observation.unsqueeze(2)  # Add a singleton dimension for num_nodes
            predicted_hidden_states.append(hidden_state.unsqueeze(1))
            # Prediction step
            hidden_state = torch.matmul(self.transition_matrix(hidden_state), observation).squeeze(-1)
            predicted_hidden_states.append(hidden_state.unsqueeze(1))
        predicted_hidden_states = torch.cat(predicted_hidden_states, dim=1)
        predicted_hidden_states = predicted_hidden_states.view(num_samples, self.num_nodes, self.num_features)  # Reshape
        predicted_observations = self.observation_matrix(predicted_hidden_states)
        # Kalman filtering
        filtered_states = []
        for t in range(num_samples):
            predicted_observation = predicted_observations[t]
            innovation = observations[t] - predicted_observation
            innovation_covariance = torch.matmul(self.observation_matrix.weight,
                                                  torch.matmul(self.process_noise_covariance, self.observation_matrix.weight.T)) + self.observation_noise_covariance
            kalman_gain = torch.matmul(self.process_noise_covariance,
                                       torch.matmul(self.observation_matrix.weight.T,
                                                    torch.inverse(innovation_covariance)))
            hidden_state = hidden_state + torch.matmul(kalman_gain, innovation.permute(0, 2, 1)).permute(0, 2, 1)
            filtered_states.append(hidden_state.unsqueeze(1))
        filtered_states = torch.cat(filtered_states, dim=1)
        return filtered_states

# Define the Graph Neural Network
class GNN(nn.Module):
    def __init__(self, num_features, hidden_dim, num_layers):
        super(GNN, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(GCNConv(num_features, hidden_dim))
        for _ in range(num_layers - 1):
            self.conv_layers.append(GCNConv(hidden_dim, hidden_dim))
        self.output_layer = nn.Linear(hidden_dim, num_features)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.conv_layers:
            x = torch.relu(conv(x, edge_index))
        x = self.output_layer(x)
        return x

# Generate random data
num_nodes = 50
num_features = np.random.randint(1, 6, size=num_nodes)  # Random number of features for each node
num_samples = 1000
observed_nodes = np.random.choice(num_nodes, size=num_nodes // 2, replace=False)  # Half of the nodes are observed
observations = np.random.randn(num_samples, num_nodes, max(num_features)).astype(np.float32)

# Initialize Kalman Filter and Graph Neural Network
kalman_filter = KalmanFilter(max(num_features), num_nodes)
gnn = GNN(max(num_features), hidden_dim=64, num_layers=3)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(list(kalman_filter.parameters()) + list(gnn.parameters()), lr=0.01)

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    
    # Kalman filtering
    filtered_states = kalman_filter(torch.tensor(observations))
    
    # Prepare graph data
    observed_edges = []
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if i in observed_nodes and j in observed_nodes:
                observed_edges.append([i, j])
    edge_index = torch.tensor(observed_edges, dtype=torch.long).t().contiguous()
    x = torch.tensor



In [ ]:
#Realization of Raindrop-based structure.
import torch
import torch.nn as nn

class SpatialInterpolation(nn.Module):
    def __init__(self):
        super(SpatialInterpolation, self).__init__()
        self.linear = nn.Linear(1, 1)  # Learnable parameter for spatial interpolation

    def forward(self, x):
        return self.linear(x)


# Define model using MultiheadAttention for attention mechanism
class AttentionMechanism(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(AttentionMechanism, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)

    def forward(self, input_data):
        # Compute attention
        attn_output, _ = self.multihead_attn(input_data, input_data, input_data)
        return attn_output


# Generate input data with time steps equal to 1000 for each feature
A_F1 = torch.rand(1000, 1, 1)  # Generate random values between 0 and 1 with shape [seq_len, batch_size, embed_dim]
A_F2 = torch.rand(1000, 1, 1)
B_F1 = torch.rand(1000, 1, 1)
C_F1 = torch.rand(1000, 1, 1)
D_F2 = torch.rand(1000, 1, 1)
D_F1_true = torch.rand(1000, 1, 1)  # Assuming D_F1_true also should be in [0, 1] range

# Define model
interp_model = SpatialInterpolation()
atten_model = AttentionMechanism(embed_dim=6, num_heads=1)  # Adjust embed_dim and num_heads as needed

# Define loss function
criterion = nn.MSELoss()

# Define optimizer
optimizer = torch.optim.Adam(list(interp_model.parameters()) + list(atten_model.parameters()), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()

    # Step 1: Spatial interpolation for B and C
    B_F2_interp = interp_model(B_F1)
    C_F2_interp = interp_model(C_F1)

    # Step 2: Attention mechanism for D_F1
    input_data = torch.cat((A_F1, A_F2, B_F1, B_F2_interp, C_F1, C_F2_interp), dim=2)  # Concatenate along the embed_dim axis
    print(input_data.shape)
    attn_output = atten_model(input_data)

    # Calculate loss
    loss = criterion(attn_output[:, :, :1], D_F1_true)  # Adjust slicing based on output dimensionality

    # Backpropagation
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Prediction
with torch.no_grad():
    input_data = torch.cat((A_F1, A_F2, B_F1, B_F2_interp, C_F1, C_F2_interp), dim=2)  # Concatenate along the embed_dim axis
    print(input_data.shape)
    attn_output = atten_model(input_data)


In [ ]:
##Realization of KF, but something happens.-----Unresolved. If I got time, try to fix it sometimes.----Try to fix it in the future.

import torch
import torch.nn as nn

class KalmanFilter(nn.Module):
    def __init__(self, num_features, num_nodes):
        super(KalmanFilter, self).__init__()
        self.num_features = num_features
        self.num_nodes = num_nodes
        self.observation_matrix = nn.Linear(num_features, num_nodes * num_features, bias=False)
        self.transition_matrix = nn.Linear(num_features, num_features, bias=False)
        self.process_noise_covariance = nn.Parameter(torch.Tensor(num_features, num_features))
        self.observation_noise_covariance = nn.Parameter(torch.Tensor(num_nodes * num_features, num_nodes * num_features))
        self.hidden_state_mean = nn.Parameter(torch.zeros(num_nodes, num_features))
        self.hidden_state_covariance = nn.Parameter(torch.eye(num_nodes * num_features))

    def forward(self, observations):
        num_samples = observations.size(0)
        predicted_hidden_states = []
        hidden_state = self.hidden_state_mean.unsqueeze(0).repeat(num_samples, 1, 1)
        for observation in observations.transpose(1, 2):  # Transpose observations
            observation = observation.unsqueeze(2)  # Add a singleton dimension for num_samples
            predicted_hidden_states.append(hidden_state.unsqueeze(1))
            # Prediction step
            print("self.transition_matrix(hidden_state).shape",self.transition_matrix(hidden_state).shape)
            print(observation.shape,"observation.shape before transpose")
            print(type(observation),"observation.shape before transpose")
            observation=torch.transpose(observation,0,2)
            print(observation.shape,"observation.shape after transpose")
            print("torch.matmul(self.transition_matrix(hidden_state), observation)",torch.matmul(self.transition_matrix(hidden_state), observation))
            hidden_state = torch.matmul(self.transition_matrix(hidden_state), observation).squeeze(-1)  # Transpose hidden_state
            predicted_hidden_states.append(hidden_state.unsqueeze(1))
        predicted_hidden_states = torch.cat(predicted_hidden_states, dim=1)
        predicted_hidden_states = predicted_hidden_states.view(num_samples, self.num_nodes, self.num_features)  # Reshape
        predicted_observations = self.observation_matrix(predicted_hidden_states)
        # Kalman filtering
        filtered_states = []
        for t in range(num_samples):
            predicted_observation = predicted_observations[t]
            innovation = observations[t] - predicted_observation
            innovation_covariance = torch.matmul(self.observation_matrix.weight,
                                                  torch.matmul(self.process_noise_covariance, self.observation_matrix.weight.T)) + self.observation_noise_covariance
            kalman_gain = torch.matmul(self.process_noise_covariance,
                                       torch.matmul(self.observation_matrix.weight.T,
                                                    torch.inverse(innovation_covariance)))
            hidden_state = hidden_state + torch.matmul(kalman_gain, innovation.permute(0, 2, 1)).permute(0, 2, 1)
            filtered_states.append(hidden_state.unsqueeze(1))
        filtered_states = torch.cat(filtered_states, dim=1)
        return filtered_states

# Define input data
num_samples = 5
num_nodes = 3
num_features = 2

# Generate random observations
observations = torch.randn(num_samples, num_features, num_nodes)  # Transpose dimensions

# Initialize the KalmanFilter model
kalman_filter = KalmanFilter(num_features, num_nodes)

# Forward pass through the KalmanFilter
filtered_states = kalman_filter(observations)

# Print the filtered states
print("Filtered states:")
print(filtered_states)




In [ ]:
observations.shape
print(type(observations))

In [ ]:
import torch

# 创建两个三维矩阵
A = torch.randn(5, 3, 2)  # 形状为 (2, 3, 4)
B = torch.randn(1, 2, 3)  # 形状为 (3, 4, 5)
print(A.shape)
# 执行矩阵乘法
result = torch.matmul(A, B)

# 打印结果的形状
print(result.shape)  # 输出: torch.Size([2, 3, 5])
